In [1]:
import numpy as np

In [496]:
def get_initial_poss(n_cells, eq_cells):
    return np.array([[*['']*i, '=', *['']*(n_cells-1-i)] for i in eq_cells])

def expand_poss(poss, col_num, explist):
    newvals = np.where(poss[:, col_num:col_num+1] == '',
                       np.expand_dims(explist, 0),
                       np.column_stack((poss[:, col_num:col_num+1], 
                                        np.full((poss.shape[0], 
                                                 explist.shape[0]-1), ''))))    
    poss_out = np.repeat(poss, np.sum(newvals != '', axis=1), axis=0)
    poss_out[:, col_num] = newvals.reshape(-1)[newvals.reshape(-1) != '']
    return poss_out

def get_filters(poss, col_num):
    filters = [np.ones((poss.shape[0]), dtype=bool)]
    if col_num > 0:
        # Back to back operators
        filters += [~(is_operator(poss, col_num) & is_operator(poss, col_num-1))]
    if 1 <= col_num <= 6:
        # Number starting with zero
        filters += [~((poss[:, col_num] == '0') & is_operator(poss, col_num-1))]
    if col_num in[3, 4, 5]:
        # Operator before equals
        filters += [~(is_operator(poss, col_num) & (poss[:, col_num+1] == '='))]
        # No operator on LHS
        filters += [~((poss[:, col_num+1] == '=') 
                      & (~np.any(np.column_stack([is_operator(poss, c) 
                                                  for c in range(col_num+1)]), axis=1)))]
    return np.all(np.column_stack(filters), axis=1)

def is_operator(poss, col_num):
    return (((poss[:, col_num] != '') 
             & (poss[:, col_num].view(np.uint32) < ord('0')))
            | (poss[:, col_num].view(np.uint32) > ord('9')))

def complete_rhs(poss, col_num):
    lhs_comp_ix = np.arange(poss.shape[0])[poss[:, col_num+1] == '=']

    lhs = poss[lhs_comp_ix][:, :col_num+1].reshape(-1).view(f'<U{col_num+1}')
    lhs_result = np.round(np.array([eval(i) for i in lhs]), 6)
    targ_res_len = poss.shape[1] - col_num - 2

    filt = np.all(np.column_stack((lhs_result >= 0,
                                   np.mod(lhs_result, 1) == 0,
                                   np.maximum(np.floor(np.log10(np.maximum(lhs_result, 1e-6)))+1, 1)
                                       == targ_res_len)),
                  axis=1)
    poss[lhs_comp_ix[filt], col_num+2:] = lhs_result[filt].astype(int)\
                                                          .astype(f'<U{targ_res_len}')\
                                                          .view('<U1')\
                                                          .reshape(-1, targ_res_len)
    return np.delete(poss, lhs_comp_ix[~filt], axis=0)

def get_all_poss():
    poss = get_initial_poss(8, [4, 5, 6])
    explist = np.arange(1, 10).astype('<U1')
    for col_num in range(6):
        poss = expand_poss(poss, col_num, explist)
        poss = poss[get_filters(poss, col_num)]
        if col_num == 0:
            explist = np.append(explist, list('0+-*/'))
        elif col_num >= 3:
            poss = complete_rhs(poss, col_num)
    return poss

In [497]:
poss = get_all_poss()
poss.shape[0], poss[:10]

100%|██████████| 6/6 [00:16<00:00,  2.72s/it]


(17723,
 array([['1', '2', '*', '9', '=', '1', '0', '8'],
        ['1', '3', '*', '8', '=', '1', '0', '4'],
        ['1', '3', '*', '9', '=', '1', '1', '7'],
        ['1', '4', '*', '8', '=', '1', '1', '2'],
        ['1', '4', '*', '9', '=', '1', '2', '6'],
        ['1', '5', '*', '7', '=', '1', '0', '5'],
        ['1', '5', '*', '8', '=', '1', '2', '0'],
        ['1', '5', '*', '9', '=', '1', '3', '5'],
        ['1', '6', '*', '7', '=', '1', '1', '2'],
        ['1', '6', '*', '8', '=', '1', '2', '8']], dtype='<U1'))

In [492]:
filt = np.all(np.column_stack([
                  np.all(poss != '4', axis=1),
                  np.all(poss != '+', axis=1),
                  np.all(poss != '9', axis=1),
                  poss[:, 0] == '1',
                  poss[:, 3] != '2',
                  np.sum(poss == '2', axis=1) >= 1,
                  poss[:, 4] != '5',
                  np.sum(poss == '5', axis=1) >= 1,
                  poss[:, 6] != '3',
                  np.sum(poss == '3', axis=1) >= 1,
                  poss[:, 5] != '=',
              ]),
              axis=1)

poss[filt]

array([['1', '1', '5', '/', '2', '3', '=', '5'],
       ['1', '1', '-', '3', '*', '2', '=', '5'],
       ['1', '3', '5', '/', '2', '7', '=', '5'],
       ['1', '3', '0', '/', '2', '6', '=', '5'],
       ['1', '3', '0', '/', '6', '5', '=', '2'],
       ['1', '3', '-', '5', '-', '2', '=', '6'],
       ['1', '3', '-', '5', '-', '6', '=', '2'],
       ['1', '3', '-', '5', '*', '2', '=', '3'],
       ['1', '3', '-', '6', '-', '2', '=', '5'],
       ['1', '3', '-', '6', '-', '5', '=', '2'],
       ['1', '5', '-', '6', '*', '2', '=', '3'],
       ['1', '5', '/', '3', '-', '2', '=', '3'],
       ['1', '5', '/', '3', '-', '3', '=', '2'],
       ['1', '6', '0', '/', '3', '2', '=', '5'],
       ['1', '7', '-', '3', '*', '5', '=', '2'],
       ['1', '7', '-', '5', '*', '3', '=', '2'],
       ['1', '0', '-', '3', '-', '2', '=', '5'],
       ['1', '0', '-', '3', '-', '5', '=', '2'],
       ['1', '0', '-', '5', '-', '2', '=', '3'],
       ['1', '0', '-', '5', '-', '3', '=', '2']], dtype='<U1')

In [503]:
# words = poss.reshape(-1).view('<U8').tolist()

# From https://github.com/pedrokkrause/Nerdle-Equations/blob/main/wordle.py:
from math import log2
from tqdm import tqdm
from collections import defaultdict
from time import sleep
from copy import deepcopy

def genmaskW(expression,correct):
    mask = ['N']*len(expression)
    count = defaultdict(lambda: 0)
    for i, x in enumerate(expression):
        if x == correct[i]:
            mask[i] = '2'
            count[x] += 1
    for i,x in enumerate(expression):
        if x not in correct:
            mask[i] = '0'
        elif x != correct[i]:
            if count[x] < correct.count(x):
                mask[i] = '1'
                count[x] += 1
            else:
                mask[i] = '0'
    return(''.join(mask))

def checkmaskW(mask,expression,correct):
    if mask == genmaskW(expression,correct):
        return(True)
    else:
        return(False)

def allmasksW(expression,dictionary):
    masks = defaultdict(lambda: 0)
    for expression2 in dictionary:
        mask = genmaskW(expression,expression2)
        masks[mask] += 1
    return(masks)

def filtermaskW(mask,expression,dictionary):
    local = [x for x in dictionary if checkmaskW(mask,expression,x)]
    return(local)

def searchW(dictionary,possible=None):
    if possible==None:
        possible = dictionary
    if len(possible) == 1 or len(dictionary) == 1:
        return([possible[0],0,possible[0] in dictionary])
    best = None
    expected = 0
    length = len(possible)
    for expression in tqdm(dictionary):
        localexp = 0
        allmaskss = allmasksW(expression,possible)
        for mask in allmaskss:
            prob = allmaskss[mask]/length
            localexp += -prob*log2(prob)
        if localexp > expected or (localexp >= expected and expression in possible):
            best = expression
            expected = localexp
            if len(possible) > 10000:
                print([best,expected,best in possible])
    return([best,expected,best in possible])

possiblewords = deepcopy(words)
print("Number of possible words:",len(possiblewords))
best = ['48-32=16']
while True:
    ans1 = input("What you inserted (enter 'b' if it was the best choice): ")
    ans2 = input("Mask: ")
    if ans1 == 'b':
        ans1 = best[0]
    possiblewords = filtermaskW(ans2,ans1,possiblewords)
    print("Number of possible words:",len(possiblewords))
    best = searchW(words,possiblewords)
    print("Best choice:",best[0])
    print("Average information:",best[1])
    if len(possiblewords) < 7:
        print("Possible words:",possiblewords)
    if ans2 == '22222222':
        break
    print("============")

Number of possible words: 17723


What you inserted (enter 'b' if it was the best choice):  14+25=39
Mask:  20011110


Number of possible words: 20


100%|██████████| 17723/17723 [00:00<00:00, 18198.02it/s]


Best choice: 13-5-6=2
Average information: 4.321928094887363


What you inserted (enter 'b' if it was the best choice):  b
Mask:  22220021


Number of possible words: 1
Best choice: 13-5*2=3
Average information: 0
Possible words: ['13-5*2=3']


What you inserted (enter 'b' if it was the best choice):  b
Mask:  22222222


Number of possible words: 1
Best choice: 13-5*2=3
Average information: 0
Possible words: ['13-5*2=3']
